# environment sanity & base config

In [1]:
%pip intall requirements.txt

# %pip install sounddevice
# %pip install python-dotenv
# %pip install scipy
# %npm install @elevenlabs/react
# %pip install websockets

ERROR: unknown command "intall" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import platform
from pathlib import Path
from datetime import datetime

In [3]:
try:
    import sounddevice as sd  #microphone capture
    sd_available = True
except Exception as e:
    sd = None
    sd_available = False

try:
    from dotenv import load_dotenv  # load .env file
    dotenv_available = True
except Exception:
    dotenv_available = False
    

print("sounddevice available:", sd_available)
print("dotenv available:", dotenv_available)

sounddevice available: True
dotenv available: True


Load and verify API KEY

In [4]:
def mask_secret(s: str, keep: int = 6) -> str:
    """mask secret string, keep first and last `keep` characters visible"""
    if not s:
        return "<EMPTY>"
    if len(s) <= keep:
        return "*" * len(s)
    return s[:keep] + "*" * (len(s) - 2*keep) + s[-keep:]

In [5]:
if dotenv_available:
    load_dotenv()  
else:
    print("[WARN] python-dotenv not found, .env loader not found）")

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY", "")

# assertion
assert ELEVENLABS_API_KEY, "ELEVENLABS_API_KEY not found, please set it in .env。"

experimental defaults

In [6]:
MODEL_ID = "scribe_v2_realtime"          # 模型 ID（model_id）
LANGUAGE_CODE = "en"                      # 语言（language_code），英文课堂/会议/面试
AUDIO_FORMAT = "pcm_16000"                # 音频格式（audio_format）：16kHz/16-bit/mono/little-endian
CHUNK_MS = 200                            # 分块时长（chunk duration ms）：初始建议 200ms
COMMIT_STRATEGY = "vad"                   # 分段策略（commit strategy）：'vad' 或 'manual'

# VAD 参数（仅在 commit_strategy = 'vad' 时有效）
VAD_SILENCE_THRESHOLD_SECS = 1.5          # 达到静音阈值后自动提交（silence threshold）
VAD_THRESHOLD = 0.4                       # 灵敏度（0.1-0.9，越低越敏感）
MIN_SPEECH_DURATION_MS = 100              # 最短语音持续（ms）
MIN_SILENCE_DURATION_MS = 100             # 最短静音持续（ms）

# WebSocket 端点（WebSocket endpoint）
WS_URL = f"wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id={MODEL_ID}"

test output dir(experimental)

In [7]:
RUNS_ROOT = Path.cwd().parent / "runs" if Path.cwd().name == "notebooks" else Path.cwd() / "runs"
RUN_DIR = RUNS_ROOT / datetime.now().strftime("%Y-%m-%d_%H%M%S")
RUN_DIR.mkdir(parents=True, exist_ok=True)

Print environment summary

In [8]:
print("== Environment Summary ==")
print(f"Project Root        : {Path.cwd().resolve().parent if Path.cwd().name == 'notebooks' else Path.cwd().resolve()}")
print(f"Python              : {sys.version.split()[0]} on {platform.system()} {platform.release()} ({platform.machine()})")
print(f".env ELEVENLABS_KEY : {mask_secret(ELEVENLABS_API_KEY, keep=6)}")
print(f"Model ID            : {MODEL_ID}")
print(f"Language Code       : {LANGUAGE_CODE}")
print(f"Audio Format        : {AUDIO_FORMAT}")
print(f"Chunk (ms)          : {CHUNK_MS}")
print(f"Commit Strategy     : {COMMIT_STRATEGY}")
if COMMIT_STRATEGY.lower() == "vad":
    print(f"  VAD silence secs  : {VAD_SILENCE_THRESHOLD_SECS}")
    print(f"  VAD threshold     : {VAD_THRESHOLD}")
    print(f"  min speech (ms)   : {MIN_SPEECH_DURATION_MS}")
    print(f"  min silence (ms)  : {MIN_SILENCE_DURATION_MS}")

print(f"WS URL              : {WS_URL}")
print(f"Run Output Dir      : {RUN_DIR.resolve()}")

if sd_available:
    try:
        ver = sd.get_portaudio_version()
        default_in = sd.default.device[0] if sd.default.device is not None else None
        print("Sounddevice         : available")
        print(f"  PortAudio version : {ver}")
        print(f"  Default input dev : {default_in}")
    except Exception as e:
        print("Sounddevice         : available but cannot query details ->", repr(e))
else:
    print("Sounddevice         : NOT available (install 'sounddevice' and ensure PortAudio is present)")

print("✅ Cell 1 successfully Initialized")

== Environment Summary ==
Project Root        : /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG
Python              : 3.10.19 on Darwin 24.6.0 (arm64)
.env ELEVENLABS_KEY : sk_c09***************************************d3c80c
Model ID            : scribe_v2_realtime
Language Code       : en
Audio Format        : pcm_16000
Chunk (ms)          : 200
Commit Strategy     : vad
  VAD silence secs  : 1.5
  VAD threshold     : 0.4
  min speech (ms)   : 100
  min silence (ms)  : 100
WS URL              : wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
Run Output Dir      : /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-13_212036
Sounddevice         : available
  PortAudio version : (1246976, 'PortAudio V19.7.0-devel, revision unknown')
  Default input dev : 2
✅ Cell 1 successfully Initialized


Microphone sanity check + PCM format validation

In [9]:
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
from pathlib import Path

Set paprameters

In [10]:
SAMPLE_RATE = 16000           # 采样率 (Hz)
CHANNELS = 1                  # 单声道 (mono)
DURATION = 5                  # 录制时长 (秒)
TEST_FILENAME = RUN_DIR / "runs/manual_test.wav"

Reccord

In [11]:
# Cell — 使用现成的测试 WAV文件（manual_test_16k.wav）做分块 + base64

import base64
import numpy as np
from scipy.io import wavfile
from pathlib import Path

project_root = Path("/Users/dwg/CODE/AI_Projects/realtime_transcriber_WG")
wav_path = project_root / "runs" / "manual_test_16k.wav"

assert wav_path.exists(), f"找不到 WAV 文件: {wav_path}"

print(f"🎧 使用测试音频: {wav_path.resolve()}")

# 读取 WAV
sr, data = wavfile.read(wav_path)
print(f"  采样率 (sample_rate): {sr} Hz")
print(f"  数据形状 (shape)    : {data.shape}, dtype={data.dtype}")

# 保证 mono（单声道）
if data.ndim == 2 and data.shape[1] > 1:
    print("  检测到多声道，转换为 mono（简单平均）")
    data_mono = data.mean(axis=1).astype(np.int16)
else:
    data_mono = data.astype(np.int16).flatten()

duration_sec = len(data_mono) / sr
print(f"  时长 (duration)     : {duration_sec:.2f} 秒")

max_val, min_val = data_mono.max(), data_mono.min()
mean_abs = np.mean(np.abs(data_mono))
print(f"  振幅范围 (max/min)  : {max_val} / {min_val}")
print(f"  平均绝对值 (mean|x|): {mean_abs:.2f}")

if mean_abs == 0:
    print("⚠️ 警告：该音频看起来是静音（mean_abs=0），转写结果可能为空，"
          "但仍可用于测试 WebSocket 流程。")



🎧 使用测试音频: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/manual_test_16k.wav
  采样率 (sample_rate): 16000 Hz
  数据形状 (shape)    : (166528,), dtype=int16
  时长 (duration)     : 10.41 秒
  振幅范围 (max/min)  : 12402 / -10723
  平均绝对值 (mean|x|): 756.69


In [12]:
# 分块（chunking），使用前面 Cell1 中定义的 CHUNK_MS
chunk_samples = int(sr * CHUNK_MS / 1000)
print(f"📦 每块样本数 (samples per chunk for {CHUNK_MS}ms): {chunk_samples}")

def generate_pcm_chunks(int16_array: np.ndarray, samples_per_chunk: int):
    n = len(int16_array)
    for start in range(0, n, samples_per_chunk):
        end = min(start + samples_per_chunk, n)
        chunk = int16_array[start:end]
        yield chunk.astype('<i2').tobytes()  # 小端 int16 PCM

chunks_bytes = list(generate_pcm_chunks(data_mono, chunk_samples))
print(f"✅ 总块数 (total chunks): {len(chunks_bytes)}")

# 预览首块的 base64
if chunks_bytes:
    first_chunk = chunks_bytes[0]
    b64_preview = base64.b64encode(first_chunk).decode("ascii")
    print(f"  首块字节长度 (bytes len)      : {len(first_chunk)}")
    print(f"  首块 base64 长度 (base64 len): {len(b64_preview)}")
    print(f"  首块 base64 预览 (前80字符)   : {b64_preview[:80]}...")
else:
    print("⚠️ 警告：没有生成任何 chunk。")

print("✅ 分块 + base64 预处理完成，可以用于后续 WebSocket 推流实验。")

📦 每块样本数 (samples per chunk for 200ms): 3200
✅ 总块数 (total chunks): 53
  首块字节长度 (bytes len)      : 6400
  首块 base64 长度 (base64 len): 8536
  首块 base64 预览 (前80字符)   : CQAAAPn////+//f/AAD+/wEABQD7//3/BAADAAIAAwAAAAQABgD8//n/AAD///7/BAABAAAA/P///wAA...
✅ 分块 + base64 预处理完成，可以用于后续 WebSocket 推流实验。


In [13]:
import asyncio
import base64
import json
from typing import List
try:
    import websockets
except ImportError as e:
    raise ImportError(
        "未找到 websockets 库，请先安装：\n"
        "  pip install websockets\n"
        "然后重新运行本 Cell。"
    ) from e

# 确保我们有 chunks_bytes 和 sr（采样率）
try:
    len(chunks_bytes)
except NameError:
    raise RuntimeError("chunks_bytes 未定义，请先运行分块 + base64 的那个 Cell。")

STREAM_SAMPLE_RATE = 16000  # 如果你以后换了采样率，这里要和音频一致

print(f"将使用 {len(chunks_bytes)} 个音频块推流，sample_rate={STREAM_SAMPLE_RATE}, chunk_ms={CHUNK_MS}ms")
print("连接 WebSocket 并开始转写...")

将使用 53 个音频块推流，sample_rate=16000, chunk_ms=200ms
连接 WebSocket 并开始转写...


In [14]:
async def send_audio(ws, chunks: List[bytes], sample_rate: int):
    """
    按固定节奏发送音频块到 WebSocket。
    模拟真实时间：每个块之间 sleep CHUNK_MS/1000 秒。
    最后发送一次 commit=True，触发 committed_transcript。
    """
    chunk_interval = CHUNK_MS / 1000.0  # 秒
    print(f"🛰 发送任务启动，每 {chunk_interval:.3f}s 发送 1 个 chunk...")

    for idx, chunk in enumerate(chunks):
        audio_b64 = base64.b64encode(chunk).decode("ascii")
        payload = {
            "message_type": "input_audio_chunk",
            "audio_base_64": audio_b64,
            "commit": False,
            "sample_rate": sample_rate,
        }
        await ws.send(json.dumps(payload))
        # 打个点方便看进度
        if (idx + 1) % 10 == 0 or idx == len(chunks) - 1:
            print(f"   -> 已发送 {idx + 1}/{len(chunks)} 块")

        await asyncio.sleep(chunk_interval)
           # 所有块发完后，稍微等一下，再发 commit
    await asyncio.sleep(0.5)
    print("📝 所有块已发送，发送最终 commit 请求...")

    final_payload = {
        "message_type": "input_audio_chunk",
        "audio_base_64": "",   # 无需再次发送音频
        "commit": True,
        "sample_rate": sample_rate,
    }
    await ws.send(json.dumps(final_payload))
    print("✅ commit 消息已发送。")

In [15]:
async def receive_transcripts(ws):
    """
    从 WebSocket 接收消息并根据 message_type 打印。
    我们主要关心：
      - session_started
      - partial_transcript
      - committed_transcript
      - committed_transcript_with_timestamps
    """
    print("📡 接收任务启动，等待转录结果...")
    try:
        while True:
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=20.0)
            except asyncio.TimeoutError:
                print("⏱ 20 秒未收到新消息，结束接收任务。")
                break

            data = json.loads(msg)
            msg_type = data.get("message_type")

            if msg_type == "session_started":
                print("🚀 会话已建立 (session_started):")
                print("   ", json.dumps(data, ensure_ascii=False)[:200], "...")
            elif msg_type == "partial_transcript":
                text = data.get("text", "")
                if text:
                    print(f"🟡 Partial: {text}")
            elif msg_type == "committed_transcript":
                text = data.get("text", "")
                print(f"🟢 Committed: {text}")
            elif msg_type == "committed_transcript_with_timestamps":
                text = data.get("text", "")
                words = data.get("words", [])
                print(f"📌 Committed + timestamps: {text}")
                if words:
                    print(f"   (示例前 3 个词): {words[:3]}")
                # 收到带时间戳的 committed，说明这一段已经完整，结束接收
                print("✅ 已收到 committed_transcript_with_timestamps，结束接收任务。")
                break
            elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                print(f"❌ 错误消息 ({msg_type}): {data}")
                # 出错时直接结束
                break
            else:
                # 其他类型消息直接打印简要信息
                print(f"ℹ️ 收到其他消息类型: {msg_type}")
                print("   ", json.dumps(data, ensure_ascii=False)[:200], "...")
    except Exception as e:
        print("❌ 接收过程中出现异常:", repr(e))

In [16]:
async def main_realtime_test():
    """
    建立 WebSocket 连接，启动发送和接收两个协程。
    """
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
    }

    print(f"🔌 连接到 WebSocket: {WS_URL}")
    async with websockets.connect(WS_URL, additional_headers=headers) as ws:
        # 先等待 session_started 消息（有些情况下服务器会主动先发）
        try:
            initial_msg = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data = json.loads(initial_msg)
            print("📥 初始消息:", data.get("message_type"), "|",
                  json.dumps(data, ensure_ascii=False)[:200], "...")
        except asyncio.TimeoutError:
            print("⚠️ 5 秒内未收到初始消息（session_started），继续发送音频。")

        # 并行执行：发送音频 + 接收转录
        sender = asyncio.create_task(send_audio(ws, chunks_bytes, STREAM_SAMPLE_RATE))
        receiver = asyncio.create_task(receive_transcripts(ws))

        # 等两个任务都结束
        await asyncio.gather(sender, receiver)

    print("🔚 WebSocket 连接已关闭。实验结束。")

In [17]:
# 在 notebook 中运行 asyncio
try:
    await main_realtime_test()  # 如果你的 Jupyter 支持直接 await（IPython 8+）
except SyntaxError:
    # 某些环境不允许在 cell 顶层用 await，就退回 asyncio.run
    import asyncio
    asyncio.run(main_realtime_test())

🔌 连接到 WebSocket: wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
📥 初始消息: session_started | {"message_type": "session_started", "session_id": "c3c3f6ddba874222a75f894f9f00ba84", "config": {"sample_rate": 16000, "audio_format": "pcm_16000", "language_code": null, "timestamps_granularity": "wo ...
🛰 发送任务启动，每 0.200s 发送 1 个 chunk...
📡 接收任务启动，等待转录结果...
   -> 已发送 10/53 块
🟡 Partial: Hello everyone
🟡 Partial: Hello everyone, I am
   -> 已发送 20/53 块
🟡 Partial: Hello everyone, I am Winston
🟡 Partial: Hello everyone, I am Winston. So nice to
   -> 已发送 30/53 块
🟡 Partial: Hello everyone, I am Winston. So nice to meet you guys.
🟡 Partial: Hello everyone I am Winston so nice to meet you guys I am.
   -> 已发送 40/53 块
🟡 Partial: Hello everyone I am Winston so nice to meet you guys I am student in
🟡 Partial: Hello everyone I am Winston. So nice to meet you guys. I am student in Kanegi Mail.
   -> 已发送 50/53 块
🟡 Partial: Hello everyone I am Winston. So nice to meet you guys. I 

In [18]:
# Cell 5 — 带事件记录 & 文件落盘的 Realtime 测试
# 功能：
#   1. 再次使用 chunks_bytes 推流到 Scribe v2 Realtime
#   2. 记录 session_started / partial / committed / timestamps 等事件
#   3. 将事件写入 JSONL 文件，将最终 committed 文本写入 .md 文件

import asyncio
import base64
import json
from typing import List, Dict, Any
from pathlib import Path

import websockets  # 确保已经 pip install websockets

# 确保有全局变量
STREAM_SAMPLE_RATE = 16000
assert 'chunks_bytes' in globals(), "请先运行分块 Cell，生成 chunks_bytes。"
assert 'RUN_DIR' in globals(), "请确保 Cell 1 已运行，定义了 RUN_DIR。"

print(f"📁 当前实验输出目录: {RUN_DIR.resolve()}")

events: List[Dict[str, Any]] = []   # 记录所有事件
final_committed_texts: List[str] = []  # 记录所有 committed 文本段

async def send_audio_recording(ws, chunks: List[bytes], sample_rate: int):
    chunk_interval = CHUNK_MS / 1000.0
    print(f"🛰 [Recorder] 每 {chunk_interval:.3f}s 发送 1 个 chunk...")

    for idx, chunk in enumerate(chunks):
        audio_b64 = base64.b64encode(chunk).decode("ascii")
        payload = {
            "message_type": "input_audio_chunk",
            "audio_base_64": audio_b64,
            "commit": False,
            "sample_rate": sample_rate,
        }
        await ws.send(json.dumps(payload))

        if (idx + 1) % 10 == 0 or idx == len(chunks) - 1:
            print(f"   -> 已发送 {idx + 1}/{len(chunks)} 块")

        await asyncio.sleep(chunk_interval)

    await asyncio.sleep(0.5)
    print("📝 [Recorder] 所有块已发送，发送最终 commit...")
    final_payload = {
        "message_type": "input_audio_chunk",
        "audio_base_64": "",
        "commit": True,
        "sample_rate": sample_rate,
    }
    await ws.send(json.dumps(final_payload))
    print("✅ [Recorder] commit 消息已发送。")


async def receive_and_record(ws):
    print("📡 [Receiver] 启动，等待转录事件...")
    try:
        while True:
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=20.0)
            except asyncio.TimeoutError:
                print("⏱ [Receiver] 20 秒未收到新消息，结束接收。")
                break

            data = json.loads(msg)
            msg_type = data.get("message_type")
            events.append(data)  # 原样记录事件

            if msg_type == "session_started":
                print("🚀 session_started")
            elif msg_type == "partial_transcript":
                text = data.get("text", "")
                if text:
                    print(f"🟡 Partial: {text}")
            elif msg_type == "committed_transcript":
                text = data.get("text", "")
                print(f"🟢 Committed: {text}")
                if text:
                    final_committed_texts.append(text)
            elif msg_type == "committed_transcript_with_timestamps":
                text = data.get("text", "")
                print(f"📌 Committed + timestamps: {text}")
                # 一般收到 timestamps 说明这一段结束，可以结束循环
                break
            elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                print(f"❌ [Receiver] 错误 ({msg_type}): {data}")
                break
            else:
                print(f"ℹ️ 其他事件: {msg_type}")
    except Exception as e:
        print("❌ [Receiver] 接收异常:", repr(e))


async def main_realtime_with_record():
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
    }

    print(f"🔌 连接 WebSocket: {WS_URL}")
    async with websockets.connect(WS_URL, additional_headers=headers) as ws:
        # 初始消息（session_started）
        try:
            initial_msg = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data = json.loads(initial_msg)
            events.append(data)
            print("📥 初始事件:", data.get("message_type"))
        except asyncio.TimeoutError:
            print("⚠️ 5 秒内未收到初始事件，继续。")

        sender = asyncio.create_task(send_audio_recording(ws, chunks_bytes, STREAM_SAMPLE_RATE))
        receiver = asyncio.create_task(receive_and_record(ws))

        await asyncio.gather(sender, receiver)

    print("🔚 WebSocket 连接已关闭。准备写入文件...")

    # 写 JSONL 事件流
    jsonl_path = RUN_DIR / "transcript_events.jsonl"
    with jsonl_path.open("w", encoding="utf-8") as f:
        for ev in events:
            f.write(json.dumps(ev, ensure_ascii=False) + "\n")
    print(f"💾 已写入事件流 JSONL: {jsonl_path.resolve()}")

    # 写最终 transcript 文本（拼接所有 committed 段）
    final_text = "\n\n".join(final_committed_texts) if final_committed_texts else ""
    md_path = RUN_DIR / "final_transcript.md"
    with md_path.open("w", encoding="utf-8") as f:
        f.write("# Final Transcript\n\n")
        if final_text:
            f.write(final_text + "\n")
        else:
            f.write("_(No committed transcript received)_\n")
    print(f"💾 已写入最终转写 Markdown: {md_path.resolve()}")

    print("✅ Cell 5 完成：本次实验的所有事件与最终文稿已落盘。")


# 运行
try:
    await main_realtime_with_record()
except SyntaxError:
    asyncio.run(main_realtime_with_record())

📁 当前实验输出目录: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-13_212036
🔌 连接 WebSocket: wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
📥 初始事件: session_started
🛰 [Recorder] 每 0.200s 发送 1 个 chunk...
📡 [Receiver] 启动，等待转录事件...
   -> 已发送 10/53 块
🟡 Partial: Hello everyone
🟡 Partial: Hello everyone, I am
   -> 已发送 20/53 块
🟡 Partial: Hello everyone, I am Winston
🟡 Partial: Hello everyone, I am Winston. So nice to
   -> 已发送 30/53 块
🟡 Partial: Hello everyone, I am Winston. So nice to meet you guys.
🟡 Partial: Hello everyone I am Winston so nice to meet you guys I am.
   -> 已发送 40/53 块
🟡 Partial: Hello everyone I am Winston so nice to meet you guys I am student in
🟡 Partial: Hello everyone I am Winston. So nice to meet you guys. I am student in Kanegi Mail.
   -> 已发送 50/53 块
🟡 Partial: Hello everyone I am Winston. So nice to meet you guys. I am a student in Kanagawa University.
   -> 已发送 53/53 块
📝 [Recorder] 所有块已发送，发送最终 commit...
✅ [Recorder] commit 

# Experiment A

In [22]:
import asyncio
import base64
import json
import time
from pathlib import Path
from typing import List, Dict, Any, Tuple

import numpy as np
from scipy.io import wavfile
try:
    from scipy.signal import resample_poly
    HAS_RESAMPLE = True
except Exception:
    HAS_RESAMPLE = False

import websockets  # 确保已安装：pip install websockets

# ==== 实验配置 ====
# 使用的 chunk 长度（毫秒）
CHUNK_MS_LIST = [100, 200, 300]

# 测试文件（不含扩展名，代码会自动加 .wav）
AUDIO_STEMS = ["01_test.wav", "02_test.wav"]

# 项目根目录 & 测试音频目录
PROJECT_ROOT = Path("/Users/dwg/CODE/AI_Projects/realtime_transcriber_WG")
AUDIO_DIR = PROJECT_ROOT / "audio_test_record"

# 输出目录沿用 RUN_DIR（在 Cell1 中已创建）
print(f"📁 实验输出目录: {RUN_DIR.resolve()}")
print(f"🎧 测试音频目录: {AUDIO_DIR.resolve()}")



📁 实验输出目录: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-13_212036
🎧 测试音频目录: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/audio_test_record


In [ ]:
# ---------- 工具函数：加载 wav, 转成 mono + 16k int16 ----------

def load_mono_16k_pcm(wav_path: Path) -> Tuple[int, np.ndarray]:
    """
    读取 wav 文件，确保输出为:
    - sample_rate = 16000
    - mono
    - dtype int16
    如果源采样率 != 16000，则使用 resample_poly 做重采样（如可用）。
    """
    assert wav_path.exists(), f"音频文件不存在: {wav_path}"
    sr, data = wavfile.read(wav_path)
    print(f"  [load] 原始采样率: {sr} Hz, shape={data.shape}, dtype={data.dtype}")

    # 转 mono
    if data.ndim == 2 and data.shape[1] > 1:
        print("  [load] 多声道 -> mono (平均)")
        data_mono = data.mean(axis=1).astype(np.int16)
    else:
        data_mono = data.astype(np.int16).flatten()

    # 如需重采样到 16k
    if sr != 16000:
        if not HAS_RESAMPLE:
            raise RuntimeError(
                f"需要将 {sr}Hz 重采样到 16000Hz，但未找到 scipy.signal.resample_poly。"
                " 请安装 scipy 或使用 ffmpeg 预先转好 16kHz wav。"
            )
        print(f"  [load] 重采样 {sr} -> 16000 Hz")
        x = data_mono.astype(np.float32) / 32768.0
        from math import gcd
        up, down = 16000, sr
        g = gcd(up, down)
        up //= g
        down //= g
        y = resample_poly(x, up, down)
        y = np.clip(y, -1.0, 1.0)
        data_16k = (y * 32767.0).astype(np.int16)
        return 16000, data_16k
    else:
        return sr, data_mono

In [26]:
def build_chunks(int16_array: np.ndarray, sample_rate: int, chunk_ms: int) -> list[bytes]:
    """按 chunk_ms 划分 int16 PCM，输出 bytes 列表（little-endian int16）。"""
    samples_per_chunk = int(sample_rate * chunk_ms / 1000)
    print(f"  [chunk] chunk_ms={chunk_ms} → 每块样本数: {samples_per_chunk}")
    chunks: list[bytes] = []
    n = len(int16_array)
    for start in range(0, n, samples_per_chunk):
        end = min(start + samples_per_chunk, n)
        chunk = int16_array[start:end]
        chunks.append(chunk.astype('<i2').tobytes())
    print(f"  [chunk] 总块数: {len(chunks)}")
    return chunks  # ✅ 关键：一定要返回 chunks

In [28]:
async def run_single_experiment(
    label: str,
    wav_path: Path,
    chunk_ms: int,
) -> Dict[str, Any]:
    """
    对单个音频文件 + 指定 chunk_ms 进行一次完整实验。
    返回摘要信息 dict，并将事件写入 jsonl。
    """
    print(f"\n==================== 实验开始: {label}, chunk_ms={chunk_ms} ====================")
    print(f"🎧 音频文件: {wav_path.name}")

    # 1. 加载并转成 mono 16k
    sr, mono_16k = load_mono_16k_pcm(wav_path)
    duration_sec = len(mono_16k) / sr
    print(f"  [info] 转换后采样率: {sr} Hz, 时长: {duration_sec:.2f} 秒, 样本数: {len(mono_16k)}")

    # 2. 构建 chunks
    chunks_bytes = build_chunks(mono_16k, sr, chunk_ms)

    # 3. 建立 WebSocket 连接并发送 + 接收
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
    }

    events: List[Dict[str, Any]] = []
    partial_events: List[Dict[str, Any]] = []
    committed_events: List[Dict[str, Any]] = []

    # 为了测延迟，用 time.monotonic 做时间戳
    t0_send = None  # 第一次发送音频块的时间
    t0_session = None  # session_started 的时间（可选）

    async def sender(ws):
        nonlocal t0_send
        interval = chunk_ms / 1000.0
        print(f"🛰 [Sender] 每 {interval:.3f}s 发送一个 chunk，总数={len(chunks_bytes)}")
        for idx, chunk in enumerate(chunks_bytes):
            if t0_send is None:
                t0_send = time.monotonic()
            t_rel = time.monotonic() - t0_send

            payload = {
                "message_type": "input_audio_chunk",
                "audio_base_64": base64.b64encode(chunk).decode("ascii"),
                "commit": False,
                "sample_rate": sr,
                "t_rel": t_rel,  # 自己加的调试字段（不会被服务端使用）
            }
            await ws.send(json.dumps(payload))
            if (idx + 1) % 10 == 0 or idx == len(chunks_bytes) - 1:
                print(f"   -> 已发送 {idx + 1}/{len(chunks_bytes)} 块, t_rel≈{t_rel:.2f}s")
            await asyncio.sleep(interval)

        await asyncio.sleep(0.5)
        t_rel = time.monotonic() - t0_send
        final_payload = {
            "message_type": "input_audio_chunk",
            "audio_base_64": "",
            "commit": True,
            "sample_rate": sr,
            "t_rel": t_rel,
        }
        await ws.send(json.dumps(final_payload))
        print(f"📝 [Sender] 已发送 commit, t_rel≈{t_rel:.2f}s")

    async def receiver(ws):
        nonlocal t0_send, t0_session
        print("📡 [Receiver] 启动，等待事件...")
        try:
            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=25.0)
                except asyncio.TimeoutError:
                    print("⏱ [Receiver] 25 秒未收到新消息，结束接收。")
                    break

                now = time.monotonic()
                data = json.loads(msg)
                msg_type = data.get("message_type")
                # 记录事件到本地列表（附加本地时间戳）
                ev = {
                    "local_time": now,
                    "since_first_send": (now - t0_send) if t0_send else None,
                    "type": msg_type,
                    "raw": data,
                }
                events.append(ev)

                if msg_type == "session_started":
                    t0_session = now
                    print("🚀 [Receiver] session_started")
                elif msg_type == "partial_transcript":
                    text = data.get("text", "")
                    if t0_send:
                        latency = now - t0_send
                    else:
                        latency = None
                    partial_events.append({
                        "time": now,
                        "since_first_send": latency,
                        "text": text,
                    })
                    if text:
                        print(f"🟡 Partial(t+{latency:.2f}s): {text}")
                elif msg_type == "committed_transcript":
                    text = data.get("text", "")
                    latency = now - t0_send if t0_send else None
                    committed_events.append({
                        "time": now,
                        "since_first_send": latency,
                        "text": text,
                    })
                    print(f"🟢 Committed(t+{latency:.2f}s): {text}")
                elif msg_type == "committed_transcript_with_timestamps":
                    text = data.get("text", "")
                    latency = now - t0_send if t0_send else None
                    print(f"📌 Committed+timestamps(t+{latency:.2f}s): {text}")
                    # 收到 timestamps 通常表示一段结束，可以结束循环
                    break
                elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                    print(f"❌ [Receiver] 错误 ({msg_type}): {data}")
                    break
                else:
                    print(f"ℹ️ [Receiver] 其他事件: {msg_type}")
        except Exception as e:
            print("❌ [Receiver] 异常:", repr(e))

    # 建立连接
    print(f"🔌 连接 WebSocket: {WS_URL}")
    async with websockets.connect(WS_URL, additional_headers=headers) as ws:
        # 先尝试接收 session_started
        try:
            msg0 = await asyncio.wait_for(ws.recv(), timeout=5.0)
            now0 = time.monotonic()
            data0 = json.loads(msg0)
            msg_type0 = data0.get("message_type")
            events.append({
                "local_time": now0,
                "since_first_send": None,
                "type": msg_type0,
                "raw": data0,
            })
            if msg_type0 == "session_started":
                t0_session = now0
                print("📥 初始事件: session_started")
            else:
                print("📥 初始事件:", msg_type0)
        except asyncio.TimeoutError:
            print("⚠️ 5 秒内未收到初始事件，直接开始发送音频。")

        sender_task = asyncio.create_task(sender(ws))
        receiver_task = asyncio.create_task(receiver(ws))
        await asyncio.gather(sender_task, receiver_task)

    print("🔚 连接结束，整理实验结果...")

    # 计算简单指标
    first_partial_time = (
        partial_events[0]["since_first_send"] if partial_events else None
    )
    first_committed_time = (
        committed_events[0]["since_first_send"] if committed_events else None
    )

    summary = {
        "label": label,
        "wav_file": str(wav_path),
        "chunk_ms": chunk_ms,
        "audio_duration_sec": duration_sec,
        "num_chunks": len(chunks_bytes),
        "num_partial_events": len(partial_events),
        "num_committed_events": len(committed_events),
        "first_partial_latency_sec": first_partial_time,
        "first_committed_latency_sec": first_committed_time,
    }

    print("📊 实验摘要:")
    for k, v in summary.items():
        print(f"  - {k}: {v}")

    # 写入文件
    safe_label = label.replace(" ", "_")
    events_path = RUN_DIR / f"expA_events_{safe_label}_chunk{chunk_ms}ms.jsonl"
    summary_path = RUN_DIR / f"expA_summary_{safe_label}_chunk{chunk_ms}ms.json"

    with events_path.open("w", encoding="utf-8") as f:
        for ev in events:
            f.write(json.dumps(ev, ensure_ascii=False) + "\n")
    with summary_path.open("w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    print(f"💾 已写入事件:  {events_path.name}")
    print(f"💾 已写入摘要:  {summary_path.name}")

    return summary

In [29]:
async def run_experiment_A():
    summaries: List[Dict[str, Any]] = []

    for stem in AUDIO_STEMS:
        # 尝试 01_test_wav 或 01_test_wav.wav
        base = AUDIO_DIR / stem
        if base.exists():
            wav_path = base
        elif (base.with_suffix(".wav")).exists():
            wav_path = base.with_suffix(".wav")
        else:
            print(f"⚠️ 跳过 {stem}：未找到 {base} 或 {base.with_suffix('.wav')}")
            continue

        for chunk_ms in CHUNK_MS_LIST:
            label = f"{stem}_chunk{chunk_ms}"
            summary = await run_single_experiment(label, wav_path, chunk_ms)
            summaries.append(summary)

    # 总表写一个汇总文件
    if summaries:
        overall_path = RUN_DIR / "expA_overall_summary.json"
        with overall_path.open("w", encoding="utf-8") as f:
            json.dump(summaries, f, ensure_ascii=False, indent=2)
        print(f"\n✅ 所有实验完成，汇总摘要已写入: {overall_path.name}")
    else:
        print("⚠️ 没有成功运行的实验，请检查 AUDIO_STEMS 和文件路径。")


# 运行实验 A
try:
    await run_experiment_A()
except SyntaxError:
    asyncio.run(run_experiment_A())


==================== 实验开始: 01_test.wav_chunk100, chunk_ms=100 ====================
🎧 音频文件: 01_test.wav
  [load] 原始采样率: 44100 Hz, shape=(1304987,), dtype=int16
  [load] 重采样 44100 -> 16000 Hz
  [info] 转换后采样率: 16000 Hz, 时长: 29.59 秒, 样本数: 473465
  [chunk] chunk_ms=100 → 每块样本数: 1600
  [chunk] 总块数: 296
🔌 连接 WebSocket: wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
📥 初始事件: session_started
🛰 [Sender] 每 0.100s 发送一个 chunk，总数=296
📡 [Receiver] 启动，等待事件...
   -> 已发送 10/296 块, t_rel≈0.91s
   -> 已发送 20/296 块, t_rel≈1.93s
🟡 Partial(t+2.12s): Required and these are some ways how
   -> 已发送 30/296 块, t_rel≈2.95s
🟡 Partial(t+3.13s): Required. And these are some ways how human,
   -> 已发送 40/296 块, t_rel≈3.97s
🟡 Partial(t+4.14s): Required. And these are some ways how human needs to be in that.
   -> 已发送 50/296 块, t_rel≈4.99s
🟡 Partial(t+5.15s): Required and these are some ways how human needs to be in that loop because we need
   -> 已发送 60/296 块, t_rel≈6.01s
🟡 Partial(t+6.15s

# Experiment B

In [30]:
# Cell B — 实验 B：VAD vs Manual Commit 对比（使用 01_test.wav, chunk_ms=200）

import asyncio
import base64
import json
import time
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import websockets

# ==== 实验配置 ====
B_AUDIO_STEM = "01_test.wav"   # 使用的测试音频文件
B_CHUNK_MS = 200              # 和实验 A 中选出的“折中值”保持一致
B_MANUAL_COMMIT_SEC = 15.0    # Manual 模式：每隔多少秒 commit 一次，可根据体验再调

PROJECT_ROOT = Path("/Users/dwg/CODE/AI_Projects/realtime_transcriber_WG")
AUDIO_DIR = PROJECT_ROOT / "audio_test_record"
wav_path = AUDIO_DIR / B_AUDIO_STEM
assert wav_path.exists(), f"找不到音频文件: {wav_path}"

print("🎧 实验 B 使用音频:", wav_path.resolve())
print("   chunk_ms =", B_CHUNK_MS, "manual_commit_sec =", B_MANUAL_COMMIT_SEC)
print("   当前 RUN_DIR:", RUN_DIR.resolve())

BASE_WS_URL = "wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime"

🎧 实验 B 使用音频: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/audio_test_record/01_test.wav
   chunk_ms = 200 manual_commit_sec = 15.0
   当前 RUN_DIR: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-13_212036


In [31]:
def prepare_chunks_for_B():
    """加载 01_test.wav -> mono 16k -> 按 B_CHUNK_MS 切块"""
    sr, mono_16k = load_mono_16k_pcm(wav_path)
    duration_sec = len(mono_16k) / sr
    print(f"  [prepare] sr={sr}, duration≈{duration_sec:.2f}s, samples={len(mono_16k)}")
    chunks_bytes = build_chunks(mono_16k, sr, B_CHUNK_MS)
    return sr, duration_sec, chunks_bytes

async def run_B_vad():
    """模式 1：VAD 自动分段（commit_strategy=vad，不手动 commit）"""
    print("\n==================== 模式 1：VAD 自动分段 ====================")
    sr, duration_sec, chunks_bytes = prepare_chunks_for_B()

    # 在 URL 中指定 VAD 参数
    vad_url = (
        BASE_WS_URL
        + "&commit_strategy=vad"
        + "&vad_silence_threshold_secs=1.2"
        + "&vad_threshold=0.4"
        + "&min_speech_duration_ms=200"
        + "&min_silence_duration_ms=200"
    )
    headers = {"xi-api-key": ELEVENLABS_API_KEY}

    events: List[Dict[str, Any]] = []
    committed_events: List[Dict[str, Any]] = []

    t0_send = None

    async def sender(ws):
        nonlocal t0_send
        interval = B_CHUNK_MS / 1000.0
        print(f"🛰 [VAD Sender] 每 {interval:.3f}s 发送 1 个 chunk，总数={len(chunks_bytes)}")
        for idx, chunk in enumerate(chunks_bytes):
            if t0_send is None:
                t0_send = time.monotonic()
            t_rel = time.monotonic() - t0_send

            payload = {
                "message_type": "input_audio_chunk",
                "audio_base_64": base64.b64encode(chunk).decode("ascii"),
                "commit": False,         # 交给 VAD 自动 commit
                "sample_rate": sr,
                "t_rel": t_rel,
            }
            await ws.send(json.dumps(payload))

            if (idx + 1) % 20 == 0 or idx == len(chunks_bytes) - 1:
                print(f"   -> 已发送 {idx + 1}/{len(chunks_bytes)} 块, t_rel≈{t_rel:.2f}s")

            await asyncio.sleep(interval)

        print("📝 [VAD Sender] 所有音频块已发送（不手动 commit）。")

    async def receiver(ws):
        print("📡 [VAD Receiver] 启动，等待 VAD 自动 commit ...")
        nonlocal t0_send
        try:
            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=30.0)
                except asyncio.TimeoutError:
                    print("⏱ [VAD Receiver] 30 秒未收到消息，结束接收。")
                    break

                now = time.monotonic()
                data = json.loads(msg)
                msg_type = data.get("message_type")

                ev = {
                    "local_time": now,
                    "since_first_send": (now - t0_send) if t0_send else None,
                    "type": msg_type,
                    "raw": data,
                }
                events.append(ev)

                if msg_type == "partial_transcript":
                    text = data.get("text", "")
                    if text:
                        print(f"🟡 [VAD] Partial: {text}")
                elif msg_type == "committed_transcript":
                    text = data.get("text", "")
                    latency = (now - t0_send) if t0_send else None
                    committed_events.append({
                        "time": now,
                        "since_first_send": latency,
                        "text": text,
                    })
                    print(f"🟢 [VAD] Committed(t+{latency:.2f}s): {text}")
                elif msg_type == "committed_transcript_with_timestamps":
                    text = data.get("text", "")
                    print(f"📌 [VAD] Committed+timestamps: {text}")
                    # 不强制 break，允许多段；这里为了实验简单，收到一次 timestamps 就结束
                    break
                elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                    print(f"❌ [VAD Receiver] 错误 ({msg_type}): {data}")
                    break
        except Exception as e:
            print("❌ [VAD Receiver] 异常:", repr(e))

    print(f"🔌 连接 VAD WebSocket: {vad_url}")
    async with websockets.connect(vad_url, additional_headers=headers) as ws:
        # 等 session_started
        try:
            msg0 = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data0 = json.loads(msg0)
            print("📥 [VAD] 初始事件:", data0.get("message_type"))
        except asyncio.TimeoutError:
            print("⚠️ [VAD] 5 秒内未收到初始事件，直接开始发送。")

        sender_task = asyncio.create_task(sender(ws))
        receiver_task = asyncio.create_task(receiver(ws))
        await asyncio.gather(sender_task, receiver_task)

    print("🔚 [VAD] 连接结束，生成摘要...")
    first_committed_time = (
        committed_events[0]["since_first_send"] if committed_events else None
    )
    summary = {
        "mode": "vad",
        "chunk_ms": B_CHUNK_MS,
        "audio_duration_sec": duration_sec,
        "num_committed_events": len(committed_events),
        "first_committed_latency_sec": first_committed_time,
        "commit_times_sec": [c["since_first_send"] for c in committed_events],
    }

    # 写文件
    summary_path = RUN_DIR / "expB_vad_summary.json"
    events_path = RUN_DIR / "expB_vad_events.jsonl"
    with summary_path.open("w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    with events_path.open("w", encoding="utf-8") as f:
        for ev in events:
            f.write(json.dumps(ev, ensure_ascii=False) + "\n")

    print("📊 [VAD] 摘要:", json.dumps(summary, ensure_ascii=False, indent=2))
    print("💾 [VAD] 已写入:", summary_path.name, ",", events_path.name)
    return summary

In [36]:
async def run_B_manual():
    """模式 2：Manual 手动分段（每 B_MANUAL_COMMIT_SEC 秒 commit）"""
    print("\n==================== 模式 2：Manual 手动分段 ====================")
    sr, duration_sec, chunks_bytes = prepare_chunks_for_B()

    manual_url = BASE_WS_URL  # 默认 commit_strategy=manual
    headers = {"xi-api-key": ELEVENLABS_API_KEY}

    events: List[Dict[str, Any]] = []
    committed_events: List[Dict[str, Any]] = []

    t0_send = None
    last_commit_audio_t = 0.0  # 上一次 commit 对应的“音频时间（秒）”

    async def sender(ws):
        nonlocal t0_send, last_commit_audio_t
        interval = B_CHUNK_MS / 1000.0
        print(f"🛰 [Manual Sender] 每 {interval:.3f}s 发送 1 个 chunk，总数={len(chunks_bytes)}")

        for idx, chunk in enumerate(chunks_bytes):
            # 初始化 t0_send
            if t0_send is None:
                t0_send = time.monotonic()

            # 当前这块音频对应的「累计音频时间」
            audio_t = (idx + 1) * interval

            payload = {
                "message_type": "input_audio_chunk",
                "audio_base_64": base64.b64encode(chunk).decode("ascii"),
                "commit": False,
                "sample_rate": sr,
                "audio_t": audio_t,
            }
            await ws.send(json.dumps(payload))

            # 每隔 B_MANUAL_COMMIT_SEC 秒发一次 commit（避免太频繁）
            if audio_t - last_commit_audio_t >= B_MANUAL_COMMIT_SEC:
                last_commit_audio_t = audio_t
                commit_payload = {
                    "message_type": "input_audio_chunk",
                    "audio_base_64": "",
                    "commit": True,
                    "sample_rate": sr,
                    "audio_t": audio_t,
                }
                await ws.send(json.dumps(commit_payload))
                print(f"📝 [Manual Sender] audio_t≈{audio_t:.1f}s 时发送 commit")

            if (idx + 1) % 20 == 0 or idx == len(chunks_bytes) - 1:
                print(f"   -> 已发送 {idx + 1}/{len(chunks_bytes)} 块, audio_t≈{audio_t:.2f}s")

            await asyncio.sleep(interval)

        # 最后再补一个 commit，确保尾部 flush
        await asyncio.sleep(0.5)
        final_audio_t = len(chunks_bytes) * interval
        final_commit = {
            "message_type": "input_audio_chunk",
            "audio_base_64": "",
            "commit": True,
            "sample_rate": sr,
            "audio_t": final_audio_t,
        }
        await ws.send(json.dumps(final_commit))
        print(f"📝 [Manual Sender] 尾部最终 commit, audio_t≈{final_audio_t:.2f}s")

    async def receiver(ws):
        print("📡 [Manual Receiver] 启动，等待 committed ...")
        nonlocal t0_send
        try:
            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=30.0)
                except asyncio.TimeoutError:
                    print("⏱ [Manual Receiver] 30 秒未收到消息，结束接收。")
                    break

                now = time.monotonic()
                data = json.loads(msg)
                msg_type = data.get("message_type")

                ev = {
                    "local_time": now,
                    "since_first_send": (now - t0_send) if t0_send else None,
                    "type": msg_type,
                    "raw": data,
                }
                events.append(ev)

                if msg_type == "partial_transcript":
                    text = data.get("text", "")
                    if text:
                        print(f"🟡 [Manual] Partial: {text}")
                elif msg_type == "committed_transcript":
                    text = data.get("text", "")
                    latency = (now - t0_send) if t0_send else None
                    committed_events.append({
                        "time": now,
                        "since_first_send": latency,
                        "text": text,
                    })
                    print(f"🟢 [Manual] Committed(t+{latency:.2f}s): {text}")
                elif msg_type == "committed_transcript_with_timestamps":
                    text = data.get("text", "")
                    print(f"📌 [Manual] Committed+timestamps: {text}")
                elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                    print(f"❌ [Manual Receiver] 错误 ({msg_type}): {data}")
                    break
        except Exception as e:
            print("❌ [Manual Receiver] 异常:", repr(e))

    print(f"🔌 连接 Manual WebSocket: {manual_url}")
    async with websockets.connect(manual_url, additional_headers=headers) as ws:
        try:
            msg0 = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data0 = json.loads(msg0)
            print("📥 [Manual] 初始事件:", data0.get("message_type"))
        except asyncio.TimeoutError:
            print("⚠️ [Manual] 5 秒内未收到初始事件，直接开始发送。")

        sender_task = asyncio.create_task(sender(ws))
        receiver_task = asyncio.create_task(receiver(ws))
        await asyncio.gather(sender_task, receiver_task)

    print("🔚 [Manual] 连接结束，生成摘要...")
    summary = {
        "mode": "manual",
        "chunk_ms": B_CHUNK_MS,
        "audio_duration_sec": duration_sec,
        "num_committed_events": len(committed_events),
        "commit_times_sec": [c["since_first_send"] for c in committed_events],
    }

    summary_path = RUN_DIR / "expB_manual_summary.json"
    events_path = RUN_DIR / "expB_manual_events.jsonl"
    with summary_path.open("w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    with events_path.open("w", encoding="utf-8") as f:
        for ev in events:
            f.write(json.dumps(ev, ensure_ascii=False) + "\n")

    print("📊 [Manual] 摘要:", json.dumps(summary, ensure_ascii=False, indent=2))
    print("💾 [Manual] 已写入:", summary_path.name, ",", events_path.name)
    return summary

In [37]:
# ======= 实验 B 调度：先跑 VAD，再跑 Manual =======
async def run_experiment_B():
    vad_summary = await run_B_vad()
    manual_summary = await run_B_manual()

    overall = {
        "vad": vad_summary,
        "manual": manual_summary,
    }
    overall_path = RUN_DIR / "expB_overall_summary.json"
    with overall_path.open("w", encoding="utf-8") as f:
        json.dump(overall, f, ensure_ascii=False, indent=2)
    print("\n✅ 实验 B 完成，总结已写入:", overall_path.name)

try:
    await run_experiment_B()
except SyntaxError:
    asyncio.run(run_experiment_B())


==================== 模式 1：VAD 自动分段 ====================
  [load] 原始采样率: 44100 Hz, shape=(1304987,), dtype=int16
  [load] 重采样 44100 -> 16000 Hz
  [prepare] sr=16000, duration≈29.59s, samples=473465
  [chunk] chunk_ms=200 → 每块样本数: 3200
  [chunk] 总块数: 148
🔌 连接 VAD WebSocket: wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime&commit_strategy=vad&vad_silence_threshold_secs=1.2&vad_threshold=0.4&min_speech_duration_ms=200&min_silence_duration_ms=200
📥 [VAD] 初始事件: session_started
🛰 [VAD Sender] 每 0.200s 发送 1 个 chunk，总数=148
📡 [VAD Receiver] 启动，等待 VAD 自动 commit ...
🟡 [VAD] Partial: Required and these are some ways how
🟢 [VAD] Committed(t+1.96s): Required and these are some ways how required and these are some ways how
🟡 [VAD] Partial: human.
   -> 已发送 20/148 块, t_rel≈3.83s
🟡 [VAD] Partial: human needs to be in that loop
🟢 [VAD] Committed(t+4.00s): human needs to be in that loop human needs to be in that loop human needs to be in that loop
🟡 [VAD] Partial: because we

# Experiment C

In [38]:
# Cell C — 实验 C：Manual commit 间隔对分段粒度的影响
# 对 01_test.wav & 02_test.wav，在不同 commit_interval 下测试分段情况

import asyncio
import base64
import json
import time
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import websockets

# ===== 实验配置 =====
C_AUDIO_STEMS = ["01_test.wav", "02_test.wav"]  # 使用两个测试音频
C_CHUNK_MS = 200                                # 固定 200ms chunk
C_COMMIT_INTERVALS = [8.0, 12.0, 20.0]          # 三种 manual commit 间隔（秒）

PROJECT_ROOT = Path("/Users/dwg/CODE/AI_Projects/realtime_transcriber_WG")
AUDIO_DIR = PROJECT_ROOT / "audio_test_record"

BASE_WS_URL = "wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime"

print("🎧 实验 C 音频目录:", AUDIO_DIR.resolve())
print("   使用音频:", C_AUDIO_STEMS)
print("   使用 commit 间隔 (sec):", C_COMMIT_INTERVALS)
print("   当前 RUN_DIR:", RUN_DIR.resolve())

🎧 实验 C 音频目录: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/audio_test_record
   使用音频: ['01_test.wav', '02_test.wav']
   使用 commit 间隔 (sec): [8.0, 12.0, 20.0]
   当前 RUN_DIR: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-13_212036


In [39]:
def prepare_chunks_for_C(wav_path: Path):
    """加载 wav -> mono 16k -> 按 C_CHUNK_MS 切块"""
    assert wav_path.exists(), f"音频文件不存在: {wav_path}"
    sr, mono_16k = load_mono_16k_pcm(wav_path)
    duration_sec = len(mono_16k) / sr
    print(f"  [prepare] {wav_path.name}: sr={sr}, duration≈{duration_sec:.2f}s, samples={len(mono_16k)}")
    chunks_bytes = build_chunks(mono_16k, sr, C_CHUNK_MS)
    return sr, duration_sec, chunks_bytes


async def run_single_manual_experiment_C(
    label: str,
    wav_path: Path,
    commit_interval_sec: float,
) -> Dict[str, Any]:
    """
    针对单个音频 + 单个 commit_interval 做一次 manual 模式实验。
    - sender 根据 commit_interval_sec 定期发 commit
    - receiver 记录 committed_transcript 事件
    - summary 返回分段统计信息
    """
    print(f"\n==================== 实验 C: {label} (commit_interval={commit_interval_sec}s) ====================")
    sr, duration_sec, chunks_bytes = prepare_chunks_for_C(wav_path)

    headers = {"xi-api-key": ELEVENLABS_API_KEY}
    manual_url = BASE_WS_URL  # 默认即 manual commit 模式

    events: List[Dict[str, Any]] = []
    committed_events: List[Dict[str, Any]] = []

    t0_send = None
    last_commit_audio_t = 0.0
    commit_audio_ts: List[float] = []  # 记录每次 commit 对应的“音频时间（秒）”

    interval = C_CHUNK_MS / 1000.0  # 每个 chunk 对应的音频长度（秒）

    async def sender(ws):
        nonlocal t0_send, last_commit_audio_t
        print(f"🛰 [Sender-{label}] 每 {interval:.3f}s 发送 1 个 chunk，总数={len(chunks_bytes)}")

        for idx, chunk in enumerate(chunks_bytes):
            if t0_send is None:
                t0_send = time.monotonic()

            audio_t = (idx + 1) * interval  # 当前块末尾对应的“音频时间”

            payload = {
                "message_type": "input_audio_chunk",
                "audio_base_64": base64.b64encode(chunk).decode("ascii"),
                "commit": False,
                "sample_rate": sr,
                "audio_t": audio_t,
            }
            await ws.send(json.dumps(payload))

            # 只要距离上一次 commit 的音频时间 >= commit_interval，就发一次 commit
            if audio_t - last_commit_audio_t >= commit_interval_sec:
                last_commit_audio_t = audio_t
                commit_audio_ts.append(audio_t)
                commit_payload = {
                    "message_type": "input_audio_chunk",
                    "audio_base_64": "",
                    "commit": True,
                    "sample_rate": sr,
                    "audio_t": audio_t,
                }
                await ws.send(json.dumps(commit_payload))
                print(f"📝 [Sender-{label}] audio_t≈{audio_t:.1f}s 时发送 commit (间隔≈{commit_interval_sec}s)")

            if (idx + 1) % 20 == 0 or idx == len(chunks_bytes) - 1:
                print(f"   -> 已发送 {idx + 1}/{len(chunks_bytes)} 块, audio_t≈{audio_t:.2f}s")

            await asyncio.sleep(interval)

        # 所有块发完后，再补一个 final commit，确保尾巴也被 flush
        await asyncio.sleep(0.5)
        final_audio_t = len(chunks_bytes) * interval
        if not commit_audio_ts or commit_audio_ts[-1] < final_audio_t:
            commit_audio_ts.append(final_audio_t)
            final_commit = {
                "message_type": "input_audio_chunk",
                "audio_base_64": "",
                "commit": True,
                "sample_rate": sr,
                "audio_t": final_audio_t,
            }
            await ws.send(json.dumps(final_commit))
            print(f"📝 [Sender-{label}] 尾部最终 commit, audio_t≈{final_audio_t:.2f}s")

    async def receiver(ws):
        nonlocal t0_send
        print(f"📡 [Receiver-{label}] 启动，等待 committed_transcript ...")
        try:
            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=30.0)
                except asyncio.TimeoutError:
                    print(f"⏱ [Receiver-{label}] 30 秒未收到消息，结束接收。")
                    break

                now = time.monotonic()
                data = json.loads(msg)
                msg_type = data.get("message_type")

                ev = {
                    "local_time": now,
                    "since_first_send": (now - t0_send) if t0_send else None,
                    "type": msg_type,
                    "raw": data,
                }
                events.append(ev)

                if msg_type == "partial_transcript":
                    text = data.get("text", "")
                    if text:
                        print(f"🟡 [Manual-{label}] Partial: {text}")
                elif msg_type == "committed_transcript":
                    text = data.get("text", "")
                    latency = (now - t0_send) if t0_send else None
                    committed_events.append({
                        "time": now,
                        "since_first_send": latency,
                        "text": text,
                    })
                    print(f"🟢 [Manual-{label}] Committed(t+{latency:.2f}s): {text}")
                elif msg_type == "committed_transcript_with_timestamps":
                    text = data.get("text", "")
                    print(f"📌 [Manual-{label}] Committed+timestamps: {text}")
                elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                    print(f"❌ [Manual-{label}] 错误 ({msg_type}): {data}")
                    break
        except Exception as e:
            print(f"❌ [Receiver-{label}] 异常:", repr(e))

    print(f"🔌 连接 WebSocket (manual): {BASE_WS_URL}")
    async with websockets.connect(BASE_WS_URL, additional_headers=headers) as ws:
        # 初始事件（session_started）
        try:
            msg0 = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data0 = json.loads(msg0)
            print(f"📥 [Manual-{label}] 初始事件:", data0.get("message_type"))
        except asyncio.TimeoutError:
            print(f"⚠️ [Manual-{label}] 5 秒内未收到初始事件，直接开始发送。")

        sender_task = asyncio.create_task(sender(ws))
        receiver_task = asyncio.create_task(receiver(ws))
        await asyncio.gather(sender_task, receiver_task)

    print(f"🔚 [Manual-{label}] 连接结束，整理分段统计...")

    # 基于 commit_audio_ts 计算每一段的音频时长
    segment_lengths_sec: List[float] = []
    if commit_audio_ts:
        prev = 0.0
        for t in commit_audio_ts:
            segment_lengths_sec.append(t - prev)
            prev = t

    if segment_lengths_sec:
        avg_seg = float(np.mean(segment_lengths_sec))
        min_seg = float(np.min(segment_lengths_sec))
        max_seg = float(np.max(segment_lengths_sec))
    else:
        avg_seg = min_seg = max_seg = 0.0

    summary = {
        "label": label,
        "mode": "manual",
        "audio_file": str(wav_path),
        "chunk_ms": C_CHUNK_MS,
        "audio_duration_sec": duration_sec,
        "commit_interval_target_sec": commit_interval_sec,
        "num_committed_events": len(committed_events),
        "num_commits_sent": len(commit_audio_ts),
        "commit_audio_times_sec": commit_audio_ts,
        "segment_lengths_sec": segment_lengths_sec,
        "avg_segment_sec": avg_seg,
        "min_segment_sec": min_seg,
        "max_segment_sec": max_seg,
    }

    # 写入文件
    safe_label = label.replace(" ", "_")
    summary_path = RUN_DIR / f"expC_manual_summary_{safe_label}.json"
    events_path = RUN_DIR / f"expC_manual_events_{safe_label}.jsonl"
    with summary_path.open("w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    with events_path.open("w", encoding="utf-8") as f:
        for ev in events:
            f.write(json.dumps(ev, ensure_ascii=False) + "\n")

    print("📊 [Manual-{label}] 摘要:")
    for k, v in summary.items():
        if k in ("segment_lengths_sec", "commit_audio_times_sec"):
            print(f"  - {k}: {len(v)} values (e.g. {str(v[:5])}...)")
        else:
            print(f"  - {k}: {v}")
    print("💾 已写入:", summary_path.name, ",", events_path.name)

    return summary


In [40]:

async def run_experiment_C():
    all_summaries: List[Dict[str, Any]] = []

    for stem in C_AUDIO_STEMS:
        wav_path = AUDIO_DIR / stem
        if not wav_path.exists():
            print(f"⚠️ 跳过 {stem}: 文件不存在 -> {wav_path}")
            continue

        for interval_sec in C_COMMIT_INTERVALS:
            label = f"{stem}_commit{interval_sec}s"
            summary = await run_single_manual_experiment_C(label, wav_path, interval_sec)
            all_summaries.append(summary)

    overall_path = RUN_DIR / "expC_overall_summary.json"
    with overall_path.open("w", encoding="utf-8") as f:
        json.dump(all_summaries, f, ensure_ascii=False, indent=2)
    print("\n✅ 实验 C 完成，总体摘要已写入:", overall_path.name)


# 运行实验 C
try:
    await run_experiment_C()
except SyntaxError:
    asyncio.run(run_experiment_C())


==================== 实验 C: 01_test.wav_commit8.0s (commit_interval=8.0s) ====================
  [load] 原始采样率: 44100 Hz, shape=(1304987,), dtype=int16
  [load] 重采样 44100 -> 16000 Hz
  [prepare] 01_test.wav: sr=16000, duration≈29.59s, samples=473465
  [chunk] chunk_ms=200 → 每块样本数: 3200
  [chunk] 总块数: 148
🔌 连接 WebSocket (manual): wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
📥 [Manual-01_test.wav_commit8.0s] 初始事件: session_started
🛰 [Sender-01_test.wav_commit8.0s] 每 0.200s 发送 1 个 chunk，总数=148
📡 [Receiver-01_test.wav_commit8.0s] 启动，等待 committed_transcript ...
🟡 [Manual-01_test.wav_commit8.0s] Partial: Required and these are some ways how
🟡 [Manual-01_test.wav_commit8.0s] Partial: Required. And these are some ways how human,
   -> 已发送 20/148 块, audio_t≈4.00s
🟡 [Manual-01_test.wav_commit8.0s] Partial: Required. And these are some ways how human needs to be in that.
🟡 [Manual-01_test.wav_commit8.0s] Partial: Required and these are some ways how human needs to 